<a href="https://colab.research.google.com/github/Matsuura699/Python_RPG/blob/main/Python_RPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python RPG 魔術師リファクタの大迷宮**

## **概要**

Pythonで短編RPGを遊ぶことができるプログラムです。

プレイヤーは#や$などの記号で表現されたダンジョンを進み、最深部にて待ち構える「魔術師リファクタ」を倒すことを目標とします。

## **操作方法**

まず、テキストボックスにプレイヤー名(6文字以内)を入力し、「決定」を押すと、ゲームを開始することができます。

ゲームは、「探索」と「戦闘」の大きく分けて二つの場面に分かれています。

### **探索**

<img src="https://drive.google.com/uc?export=view&id=1DiYRjYsqt_SA0xTjFbbhSVa5ZHtl3veo" width=25%>

「探索」は、主人公を操作し、次の階層に進むことを目指す場面です。画面上部には、現在の階層、名前、LV、HP(後述)が表示されています。画面中央部には、ダンジョンのマップが表示されています。

各記号の表すものは以下の通りです。

- **@** ... **主人公** 画面下部の矢印のボタンを押すことで、指定した方向に動かすことができます。

- **&** ... **階段** 主人公が到達することで、次の階層に進むことができます。

- **$** ... **敵** 主人公に向かって近づき、ぶつかると戦闘が開始します。

- **#** ... **壁** 壁のある場所は、主人公も敵も通ることができません。


また、「状態」を押すと、主人公のパラメータなどを表示する画面に移行します。

<img src="https://drive.google.com/uc?export=view&id=1qTZSjRn-0wIWKi54NZuwFYuudLGanbdj" width=40%>

各パラメータの表すものは以下の通りです。

- **LV** ... 主人公の強さを表し、EXPを獲得することで少しずつ上昇していきます。LVが高くなると、他のパラメータが上昇したり、新しいスキルを使用できるようになります。
- **HP** ... 敵の攻撃を受けると減少し、0になるとGAME OVERになります。
- **SP** ... 攻撃やカウンターによって回復し、スキルを使用するとスキルに応じたSPが減少します。
- **ATK** ... このパラメータが高いほど、敵に与えるダメージが増加します。
- **DEF** ... このパラメータが高いほど、敵から受けるダメージが減少します。
- **EXP** ... 敵を倒すことで少しづつ貯まっていき、一定の値に達するとLVが上昇します。

「スキル」の項目には主人公が使うことのできるスキルの一覧とその説明が表示されており、矢印のボタンを押すことでカーソルを動かし、スキルを選択することができます。このうち、回復を行うスキルは「決定」を押すことで使用することができます。

「戻る」を押すと、元の探索画面に戻ります。

### **戦闘**

<img src="https://drive.google.com/uc?export=view&id=1TeDl30_BUph3JQjEoixI8-L9259wjEse" width=30%>

「戦闘」は、敵を攻撃し、敵のHPを0にすることを目指す場面です。戦闘が開始した後、「攻撃」「スキル」「逃げる」の中からコマンドを選択できます。
- **攻撃** ... 攻撃力に応じたダメージを敵に与えます。
- **スキル** ... 主人公が覚えているスキルの中から一つ選択し、SPを消費して使用します。
- **逃げる** ... 敵から逃走し戦闘を終了させます。しかし、LVが低ければ失敗することもあります。

相手のHPを0にすると敵に応じたEXPを獲得し、「探索」に戻ります。一方、主人公のHPが0になるとGAME OVERになるため、細心の注意を払いましょう。

### **注意点**

ボタンの連打は誤作動の原因となります。一度ずつ確実に押しましょう。

## **制作者のコメント**

敵のステータスは高めですが、こちらの獲得できるEXPも多めに設定してあります。しっかりとLVを上げながら進めれば、十分クリアできるはずです。また、最大LVは15です。

攻略のコツは、SPをしっかりと確保することです。特に、ヒール系のスキルはいつでも使えるように心がけておきましょう。

ゲーム制作には不慣れですが、何とかゲームらしいものを作ってみようと奮闘してみました。結果的にコードは1000行を超え、予想していたよりもかなり多くの作業時間を費やしてしまいました。ゲームバランスにはそれなりに気を配っていますが、如何せんデバッガーが私しかいないので、少し不十分な可能性があります。しかしRPGにおいては、大味なゲームバランスもまたゲームの醍醐味ということで、それも含めて楽しんでいただけたら良いなと思います。

## **作成時間**

作成時間：**約32時間**

In [ ]:
%reset -f
import random as r
from IPython.display import clear_output
import ipywidgets as widgets
import time as t

#初期設定
width = 15
height = 15
floor = 1
main_map = [[0 for i in range(width)] for i in range(height)]
scene = 'start'
cursor_locate = 0

player_status = {
  'name': 'name',
  'lv': 1,
  'hp_max': 25,
  'hp': 25,
  'sp_max': 100,
  'sp': 0,
  'atk': 8,
  'def': 5,
  'exp': 0,
  'need_exp': 8,
  'skills': ['カウンター'],
  'learn_lv': [3,5,9,11],
  'learn_skill': ['ヒール','全力斬り','メガヒール','ドラゴンスレイヤー'],
  'skills_sp': [0,7,10,21,30]
}

skills_explain = ['相手の攻撃を防御しつつ攻撃する。SPがたまりやすい','HPを回復する。','敵を強い力で斬る。','HPを大きく回復する。','敵をとても強い力で斬る。ドラゴンに良く効く。']

enemy_status_list = {
  'name': ['プリントスライム','ストリウルフ','イントゴースト','シャープラミア','イフガーゴイル','ライブラサラマンダー','ファンクタウルス','リストリス','ディクドラグーン','ヴァリアブル・ゴーレム','魔術師リファクタ','魔竜パイソニック'],
  'lv':  [1,1, 2, 6, 8,10,16,20,22,20,30,30],
  'hp':  [6,8,11,21,26,31,37,45,50,50,150,200],
  'atk': [4,6, 9,14,19,25,30,36,40,30,45,55],
  'def': [1,2, 3,11,15,19,23,30,35,30,40,50],
  'exp': [2,4, 7,12,18,24,30,38,45,50, 0, 0],
  'type': ['none','bea','none','bea','gol','dra','gol','bea','dra','gol','wiz','dra',]
}

def dungeon_set():
  global player_x,player_y,goal_x,goal_y,main_map
# 外壁を作成する
  for i in range(width):
    main_map[i][0] = 1
    main_map[i][height-1] = 1
  for i in range(height):
    main_map[0][i] = 1
    main_map[width-1][i] = 1

# 壁を配置する
  for i in range(height//2-1):
    for j in range(width//2-1):
      wall_i = r.randint(0,2)
      wall_j = r.randint(0,1)
      put_i = (i+1)*2
      put_j = (j+1)*2
      if wall_i == 0:
        if wall_j == 0:
          main_map[put_i][put_j] = 1
        elif wall_j == 1 and j*2+1 < height:
          main_map[put_i][put_j+1] = 1
      elif wall_i == 1 and i*2+1 < height:
        if wall_j == 0:
          main_map[put_i+1][put_j] = 1
        elif wall_j == 1 and j*2+1 < height:
          main_map[put_i+1][put_j+1] = 1
      elif wall_i == 2 and i*2+1 < height:
        if wall_j == 0:
          main_map[put_i-1][put_j] = 1
        elif wall_j == 1 and j*2+1 < height:
          main_map[put_i-1][put_j+1] = 1

# 壁の集まった部分を埋める
  for i in range(1,height-1):
    for j in range(1,width-1):
      if sum([main_map[i+1][j] == 1 , main_map[i][j+1] == 1 , main_map[i-1][j] ==1 , main_map[i][j-1] == 1]) >2:
        main_map[i][j] = 1

# スタート地点を決める
  player_x = r.randint(1,width-2)
  player_y = r.randint(1,height-2)
  while main_map[player_y][player_x] == 1:
    player_x = r.randint(1,width-2)
    player_y = r.randint(1,height-2)
  main_map[player_y][player_x] = 2

# 階段の位置を決める
  goal_x = r.randint(1,width-2)
  goal_y = r.randint(1,height-2)
  while (goal_x-3 <= player_x <= goal_x+3 and goal_y-3 <= player_y <= goal_y+3) or main_map[goal_y][goal_x] == 1:
    goal_x = r.randint(1,width-2)
    goal_y = r.randint(1,height-2)
  main_map[goal_y][goal_x] = 3

# 敵の位置を決める
def enemy_set():
  global enemy1_x,enemy1_y,enemy2_x,enemy2_y,enemy3_x,enemy3_y,main_map
  enemy1_x = r.randint(1,width-2)
  enemy1_y = r.randint(1,width-2)
  while main_map[enemy1_y][enemy1_x] == 1 or main_map[enemy1_y][enemy1_x] == 2 or main_map[enemy1_y][enemy1_x] == 3:
    enemy1_x = r.randint(1,width-2)
    enemy1_y = r.randint(1,width-2)
  main_map[enemy1_y][enemy1_x] = 4

  enemy2_x = r.randint(1,width-2)
  enemy2_y = r.randint(1,width-2)
  while main_map[enemy2_y][enemy2_x] == 1 or main_map[enemy2_y][enemy2_x] == 2 or main_map[enemy2_y][enemy2_x] == 3 or main_map[enemy2_y][enemy2_x] == 4:
    enemy2_x = r.randint(1,width-2)
    enemy2_y = r.randint(1,width-2)
  main_map[enemy2_y][enemy2_x] = 4

  enemy3_x = r.randint(1,width-2)
  enemy3_y = r.randint(1,width-2)
  while main_map[enemy3_y][enemy3_x] == 1 or main_map[enemy3_y][enemy3_x] == 2 or main_map[enemy3_y][enemy3_x] == 3 or main_map[enemy3_y][enemy3_x] == 4:
    enemy3_x = r.randint(1,width-2)
    enemy3_y = r.randint(1,width-2)
  main_map[enemy3_y][enemy3_x] = 4

# 敵の再配置を行う際に、敵を一度消す
def enemy_remove():
  main_map[enemy1_y][enemy1_x] = 0
  main_map[enemy2_y][enemy2_x] = 0
  main_map[enemy3_y][enemy3_x] = 0

# 敵の動きを制御する
def enemy_move(_):
  global enemy1_x,enemy1_y,enemy2_x,enemy2_y,enemy3_x,enemy3_y,player_x,player_y,main_map
  if floor != 5 and floor != 10:
    if enemy1_x == player_x:
      direction = 1
    elif enemy1_y == player_y:
      direction = 0
    else:
      direction = r.randint(0,1)
    main_map[enemy1_y][enemy1_x] = 0
    if direction == 0:
      if enemy1_x < player_x:
        if main_map[enemy1_y][enemy1_x+1] == 0:
          enemy1_x += 1
        elif main_map[enemy1_y][enemy1_x+1] == 2:
          encount()
          return True
      elif enemy1_x > player_x:
        if main_map[enemy1_y][enemy1_x-1] == 0:
          enemy1_x -= 1
        elif main_map[enemy1_y][enemy1_x-1] == 2:
          encount()
          return True
    else:
      if enemy1_y < player_y:
        if main_map[enemy1_y+1][enemy1_x] == 0:
          enemy1_y += 1
        elif main_map[enemy1_y+1][enemy1_x] == 2:
          encount()
          return True
      elif enemy1_y > player_y:
        if main_map[enemy1_y-1][enemy1_x] == 0:
          enemy1_y -= 1
        elif main_map[enemy1_y-1][enemy1_x] == 2:
          encount()
          return True
    main_map[enemy1_y][enemy1_x] = 4

    if enemy1_x == player_x:
      direction = 1
    elif enemy1_y == player_y:
      direction = 0
    else:
      direction = r.randint(0,1)
    main_map[enemy2_y][enemy2_x] = 0
    if direction == 0:
      if enemy2_x < player_x:
        if main_map[enemy2_y][enemy2_x+1] == 0:
          enemy2_x += 1
        elif main_map[enemy2_y][enemy2_x+1] == 2:
          encount()
          return True
      elif enemy2_x > player_x:
        if main_map[enemy2_y][enemy2_x-1] == 0:
          enemy2_x -= 1
        elif main_map[enemy2_y][enemy2_x-1] == 2:
          encount()
          return True
    else:
      if enemy2_y < player_y:
        if main_map[enemy2_y+1][enemy2_x] == 0:
          enemy2_y += 1
        elif main_map[enemy2_y+1][enemy2_x] == 2:
          encount()
          return True
      elif enemy2_y > player_y:
        if main_map[enemy2_y-1][enemy2_x] == 0:
          enemy2_y -= 1
        elif main_map[enemy2_y-1][enemy2_x] == 2:
          encount()
          return True
    main_map[enemy2_y][enemy2_x] = 4

    if enemy1_x == player_x:
      direction = 1
    elif enemy1_y == player_y:
      direction = 0
    else:
      direction = r.randint(0,1)
    main_map[enemy3_y][enemy3_x] = 0
    if direction == 0:
      if enemy3_x < player_x:
        if main_map[enemy3_y][enemy3_x+1] == 0:
          enemy3_x += 1
        elif main_map[enemy3_y][enemy3_x+1] == 2:
          encount()
          return True
      elif enemy3_x > player_x:
        if main_map[enemy3_y][enemy3_x-1] == 0:
          enemy3_x -= 1
        elif main_map[enemy3_y][enemy3_x-1] == 2:
          encount()
          return True
    else:
      if enemy3_y < player_y:
        if main_map[enemy3_y+1][enemy3_x] == 0:
          enemy3_y += 1
        elif main_map[enemy3_y+1][enemy3_x] == 2:
          encount()
          return True
      elif enemy3_y > player_y:
        if main_map[enemy3_y-1][enemy3_x] == 0:
          enemy3_y -= 1
        elif main_map[enemy3_y-1][enemy3_x] == 2:
          encount()
          return True
    main_map[enemy3_y][enemy3_x] = 4
    return False

# ダンジョンを表示する
def dungeon_print():
  print(f'{floor}F  {player_status["name"]} Lv{player_status["lv"]}  HP {player_status["hp"]}/{player_status["hp_max"]}')
  for i in range (len(main_map)):
    if floor == 5 or floor == 10:
      for k in range(4):
        print(' ',end = ' ')
    for j in range (len(main_map)):
     if player_y-2 <= i <= player_y+2 and player_x-2 <= j <= player_x+2:
        if main_map[i][j] == 0:
          print(' ',end = ' ')
        elif main_map[i][j] == 1:
          print('#',end = ' ')
        elif main_map[i][j] == 2:
          print('@',end = ' ')
        elif main_map[i][j] == 3:
          print('&',end = ' ')
        elif main_map[i][j] == 4:
          if floor != 5 or floor != 10:
            print('$',end = ' ')
        elif main_map[i][j] == 5:
          print('$',end = ' ')
     else:
       print('.',end = ' ')
    print()
  display(dungeon_button)

# 階層が変わるときの制御を行う
def stage():
  global main_map,floor,player_x,player_y,enemy1_x,enemy1_y,enemy2_x,enemy2_y,enemy3_x,enemy3_y
  floor += 1
  clear_output()
  if floor == 5 or floor == 10:
    main_map = [
    [1,1,1,1,1,1,1],
    [1,1,0,3,0,1,1],
    [1,0,0,0,0,0,1],
    [1,1,1,5,1,1,1],
    [1,0,0,0,0,0,1],
    [1,1,0,0,0,1,1],
    [1,1,1,1,1,1,1],
    ]
    player_x = 3
    player_y = 5
    main_map[player_y][player_x] = 2
    enemy1_x = 0
    enemy1_y = 0
    enemy2_x = 0
    enemy2_y = 0
    enemy3_x = 0
    enemy3_y = 0
  elif floor == 11:
    print(f'かくして、{player_status["name"]}は、魔術師リファクタに打ち勝った---')
    t.sleep(3)
    print()
    print('Thank You for Playing!!!')
    return
  else:
    main_map = [[0 for i in range(width)] for i in range(height)]
    dungeon_set()
    enemy_set()
  for i in range(15):
    for j in range(15):
      print('@',end = ' ')
    print()
  t.sleep(1)
  clear_output()
  dungeon_print()

# 敵と遭遇したときの制御を行う
def encount():
  global enemy_status,scene,power,enemy_number
  clear_output()
  scene = 'battle'
  power = False
  if floor == 1 or floor == 2:
    enemy_number = r.randint(0,2)
  elif floor == 3 or floor == 4:
    enemy_number = r.randint(2,4)
  elif floor == 6 or floor == 7:
    enemy_number = r.randint(4,6)
  elif floor == 8 or floor == 9:
    enemy_number = r.randint(6,8)
  enemy_status = {
  'name': enemy_status_list["name"][enemy_number],
  'lv': enemy_status_list["lv"][enemy_number],
  'hp': enemy_status_list["hp"][enemy_number],
  'atk': enemy_status_list["atk"][enemy_number],
  'def': enemy_status_list["def"][enemy_number],
  'exp': enemy_status_list["exp"][enemy_number],
  'type': enemy_status_list["type"][enemy_number],
  'boss': False
}
  for i in range (15):
    for j in range (15):
      print('$',end = ' ')
    print()
  t.sleep(1)
  clear_output()
  print(f'{enemy_status["name"]}があらわれた！')
  t.sleep(1.5)
  battle()

# ボスに遭遇した時の出力をする
def boss_check():
  global enemy_number
  clear_output()
  for i in range (15):
    for j in range (15):
      print('$',end = ' ')
    print()
  t.sleep(1)
  clear_output()
  if floor == 5:
    enemy_number = 9
    clear_output()
    print('「ココハ...トオサナイ...」')
    t.sleep(2)
  else:
    enemy_number = 10
    print('「よくぞここまでたどり着いたな...')
    t.sleep(2)
    print('　だが、我を倒すことなど決してできぬ...')
    t.sleep(2)
    print('　我が魔術の前に、ひれ伏すがよい...！」')
    t.sleep(2)
  boss_encount()

def boss_encount():
  global enemy_status,scene,main_map,power
  clear_output()
  scene = 'battle'
  power = False
  enemy_status = {
  'name': enemy_status_list["name"][enemy_number],
  'lv': enemy_status_list["lv"][enemy_number],
  'hp': enemy_status_list["hp"][enemy_number],
  'atk': enemy_status_list["atk"][enemy_number],
  'def': enemy_status_list["def"][enemy_number],
  'exp': enemy_status_list["exp"][enemy_number],
  'type': enemy_status_list["type"][enemy_number],
  'boss': True
}
  print(f'{enemy_status["name"]}が立ちふさがった！')
  t.sleep(1.5)
  main_map[3][3] = 0
  battle()

# 戦闘のボタンなどを出す
def battle():
  clear_output()
  print(f'{player_status["name"]} Lv{player_status["lv"]}  HP {player_status["hp"]}/{player_status["hp_max"]} SP {player_status["sp"]}/{player_status["sp_max"]}')
  print()
  print(enemy_status["name"])
  print()
  print('コマンドを選択してください')
  display(buttlebutton)

# 戦闘に勝った時の出力をする
def battle_win():
  global player_status,enemy_number
  clear_output()
  print(f'{player_status["name"]}の勝利！')
  t.sleep(2)
  if player_status["lv"] < 15:
    player_status["exp"] += enemy_status["exp"]
    print()
    print(f'{player_status["name"]}は{enemy_status["exp"]}のEXPを得た！')
    t.sleep(2)
    if player_status["need_exp"] <= player_status["exp"]:
      lv_up()
  if enemy_number == 10:
    clear_output()
    print('「まだだ...まだ終わってはおらぬ...')
    t.sleep(2)
    print('　我の真の姿を見せてやろう...最終決戦だ...！」')
    t.sleep(2)
    enemy_number = 11
    clear_output()
    for i in range (15):
      for j in range (15):
        print('$',end = ' ')
      print()
    t.sleep(1)
    boss_encount()
    return
  elif enemy_number == 11:
    clear_output()
    print('「グオオ...！')
    t.sleep(2)
    print('　よくぞ我を打ち倒したな...見事であった')
    t.sleep(2)
    print('　それから...画面の前の諸君')
    t.sleep(2)
    print('　プログラミング初心者の幼稚なゲームをここまで遊んでくれたこと、心より感謝を述べる')
    t.sleep(3)
    print('　...本当に、ありがとう...')
    t.sleep(2)
  battle_end()

# 戦闘に敗北した際の出力をする
def battle_defeat():
  clear_output()
  print(f'かくして、{player_status["name"]}は道半ばにして倒れた---')
  t.sleep(3)
  print()
  print('GAME OVER')
  return

# 戦闘が終わった時の出力をする(逃げた時も含む)
def battle_end():
  global scene
  clear_output()
  scene = 'dungeon'
  for i in range(15):
    for j in range(15):
      print('@',end = ' ')
    print()
  t.sleep(1)
  clear_output()
  if floor != 5 and floor != 10:
    enemy_remove()
    enemy_set()
  dungeon_print()

# LVが上がった時の出力をする
def lv_up():
  global player_status
  player_status["need_exp"] += int(8 * 1.3 ** player_status["lv"])
  status_up_dict = {'HP': int(5 * (1.02 ** ((player_status["lv"]-1)))) ,
                    'ATK': int(3 * (1.01 ** ((player_status["lv"]-1)))),
                    'DEF': int(2 * (1.035 ** ((player_status["lv"]-1))))}
  player_status["lv"] += 1
  player_status["hp_max"] += status_up_dict['HP']
  player_status["hp"]  = player_status["hp_max"]
  player_status["atk"] += status_up_dict['ATK']
  player_status["def"] += status_up_dict['DEF']
  clear_output()
  print(f'{player_status["name"]}はLVが上がった！')
  t.sleep(2)
  print()
  for i,j in status_up_dict.items():
    print(f'{i}が{j}上がった！')
    t.sleep(0.5)
  if player_status["lv"] in player_status["learn_lv"]:
    skill_index = player_status["learn_lv"].index(player_status["lv"])
    player_status["skills"].append(player_status["learn_skill"][skill_index])
    print()
    print(f'{player_status["learn_skill"][skill_index]}を覚えた！')
    t.sleep(0.5)
  print()
  print('さらにHPが全回復！')
  t.sleep(3)
  if player_status["lv"] < 15:
    if player_status["need_exp"] <= player_status["exp"]:
      lv_up()

# 敵が攻撃するときの制御をする
def enemy_atk():
  global player_status,enemy_damage,power
  enemy_damage = int((enemy_status["atk"]/2 - player_status["def"]/4) * r.randint(8,12) / 10)
  if enemy_damage < 1:
    enemy_damage = 1
  player_status["hp"] -= enemy_damage
  print(f'{enemy_status["name"]}の攻撃！')
  t.sleep(0.5)
  print(f'{player_status["name"]}に{enemy_damage}のダメージ！')
  t.sleep(1)
  if power:
    enemy_status["atk"] /= 2
    power = False
  if player_status["hp"] <= 0:
    return True
  return False

# 敵が行動するときの制御をする
def enemy_act():
  global player_status,enemy_damage,power
  act = r.randint(0,2)
  if act != 0:
    return enemy_atk()
  else:

    if enemy_status["type"] == 'bea':
      print(f'{enemy_status["name"]}の素早い攻撃！')
      for i in range(2):
        enemy_damage = int((enemy_status["atk"]/2 - player_status["def"]/4) * 0.7 * r.randint(8,12) / 10)
        if enemy_damage < 1:
          enemy_damage = 1
        player_status["hp"] -= enemy_damage
        t.sleep(0.5)
        print(f'{player_status["name"]}に{enemy_damage}のダメージ！')
      t.sleep(1)
      if player_status["hp"] <= 0:
        return True
      return False

    elif enemy_status["type"] == 'gol':
      if power:
        enemy_atk()
      else:
        enemy_status["atk"] *= 2
        power = True
        enemy_damage = 0
        print(f'{enemy_status["name"]}は力をためた！')
        t.sleep(1)
        return False

    elif enemy_status["type"] == 'dra':
      enemy_damage = int((enemy_status["atk"]/2 - player_status["def"]/4) * 1.6 * r.randint(8,12) / 10)
      if enemy_damage < 1:
        enemy_damage = 1
      player_status["hp"] -= enemy_damage
      print(f'{enemy_status["name"]}は燃え盛る息を吐いた！')
      t.sleep(0.5)
      print(f'{player_status["name"]}に{enemy_damage}のダメージ！')
      t.sleep(1)
      if player_status["hp"] <= 0:
        return True
      return False

    elif enemy_status["type"] == 'wiz':
     enemy_damage = int((enemy_status["atk"]/2 - player_status["def"]/4) * 1.7 * r.randint(8,12) / 10)
     if enemy_damage < 1:
       enemy_damage = 1
     player_status["hp"] -= enemy_damage
     print(f'{enemy_status["name"]}はフォーマを唱えた！')
     t.sleep(0.5)
     print(f'{player_status["name"]}に{enemy_damage}のダメージ！')
     t.sleep(1)
     if player_status["hp"] <= 0:
       return True
     return False

    else:
      return enemy_atk()

# ボタンを定義する
button_size = widgets.Layout(width = '60px', height = '60px')

up = widgets.Button(description="↑",layout = button_size)
down = widgets.Button(description="↓",layout = button_size)
left = widgets.Button(description="←",layout = button_size)
right = widgets.Button(description="→",layout = button_size)
status = widgets.Button(description="状態",layout = button_size)
enter = widgets.Button(description="はい",layout = button_size)
attack = widgets.Button(description="攻撃",layout = button_size)
skill = widgets.Button(description="スキル",layout = button_size)
run = widgets.Button(description="逃げる",layout = button_size)
back = widgets.Button(description="戻る",layout = button_size)
enter = widgets.Button(description="決定",layout = button_size)
space = widgets.Box(layout = widgets.Layout(width = '64px'))

# 必要なものを横に並べる
dungeon_button = widgets.VBox([widgets.HBox([status, up]),widgets.HBox([left, down, right])])
buttlebutton = widgets.HBox([attack, skill, run])
selectbutton = widgets.VBox([widgets.HBox([space, up]),widgets.HBox([enter, down,back])])
startbutton = widgets.HBox([space,space,enter])

# ↑ボタン
def press_up(b):
  global player_y,main_map,cursor_locate
  if scene == 'dungeon':
    if main_map[player_y-1][player_x] == 0:
      main_map[player_y][player_x] = 0
      player_y -= 1
      main_map[player_y][player_x] = 2
      is_encounted = enemy_move(0)
      if is_encounted:
        return
      clear_output()
      dungeon_print()
    elif main_map[player_y-1][player_x] == 3:
      stage()
    elif main_map[player_y-1][player_x] == 4:
      encount()
    elif main_map[player_y-1][player_x] == 5:
      boss_check()
  elif scene == 'battle':
    if cursor_locate == 0:
      cursor_locate = len(player_status["skills"])-1
    else:
      cursor_locate -= 1
    press_skill(b)
  else:
    if cursor_locate == 0:
      cursor_locate = len(player_status["skills"])-1
    else:
      cursor_locate -= 1
    press_status(b)

# ↓ボタン
def press_down(b):
  global player_y,main_map,cursor_locate
  if scene == 'dungeon':
    if main_map[player_y+1][player_x] == 0:
      main_map[player_y][player_x] = 0
      player_y += 1
      main_map[player_y][player_x] = 2
      is_encounted = enemy_move(0)
      if is_encounted:
        return
      clear_output()
      dungeon_print()
    elif main_map[player_y+1][player_x] == 3:
      stage()
    elif main_map[player_y+1][player_x] == 4:
      encount()
    elif main_map[player_y+1][player_x] == 5:
      boss_encount()
  elif scene == 'battle':
    if cursor_locate == len(player_status["skills"])-1:
      cursor_locate = 0
    else:
      cursor_locate += 1
    press_skill(b)
  else:
    if cursor_locate == len(player_status["skills"])-1:
      cursor_locate = 0
    else:
      cursor_locate += 1
    press_status(b)

# ←ボタン
def press_left(b):
  global player_x,main_map
  if main_map[player_y][player_x-1] == 0:
    main_map[player_y][player_x] = 0
    player_x -= 1
    main_map[player_y][player_x] = 2
    is_encounted = enemy_move(0)
    if is_encounted:
      return
    clear_output()
    dungeon_print()
  elif main_map[player_y][player_x-1] == 3:
    stage()
  elif main_map[player_y][player_x-1] == 4:
    encount()
  elif main_map[player_y][player_x-1] == 5:
    boss_encount()

# →ボタン
def press_right(b):
  global player_x,main_map
  if main_map[player_y][player_x+1] == 0:
    main_map[player_y][player_x] = 0
    player_x += 1
    main_map[player_y][player_x] = 2
    is_encounted = enemy_move(0)
    if is_encounted:
      return
    clear_output()
    dungeon_print()
  elif main_map[player_y][player_x+1] == 3:
    stage()
  elif main_map[player_y][player_x+1] == 4:
    encount()
  elif main_map[player_y][player_x+1] == 5:
    boss_encount()

# 状態ボタン
def press_status(b):
  global scene
  scene = 'status'
  clear_output()
  print(f'{player_status["name"]} LV{player_status["lv"]}')
  print(f'HP   {player_status["hp"]} / {player_status["hp_max"]}')
  print(f'SP   {player_status["sp"]} / {player_status["sp_max"]}')
  print(f'ATK  {player_status["atk"]}')
  print(f'DEF  {player_status["def"]}')
  if player_status["lv"] == 15:
    print('EXP  MAX')
  else:
    print(f'EXP  {player_status["exp"]} / {player_status["need_exp"]}')
  print()
  print('スキル')
  for i in range(len(player_status["skills"])):
    if i == cursor_locate:
      print('>',end='')
    else:
      print(' ',end='')
    print(f'SP{player_status["skills_sp"][i]}',end=' ')
    print(player_status["skills"][i])
  print()
  print(skills_explain[cursor_locate])
  display(selectbutton)

# 攻撃ボタン
def press_attack(b):
  global enemy_status
  clear_output()
  player_damage = int((player_status["atk"]/2 - enemy_status["def"]/4) * r.randint(8,12) / 10)
  if player_damage < 1:
    player_damage = 1
  enemy_status["hp"] -= player_damage
  print(f'{player_status["name"]}の攻撃！')
  t.sleep(0.5)
  print(f'{enemy_status["name"]}に{player_damage}のダメージ！')
  if player_status['sp_max'] > player_status['sp']:
    sp_heal = int(player_damage * r.randint(8,12) / 10)
    player_status["sp"] += sp_heal
    if player_status['sp_max'] <= player_status['sp']:
      player_status['sp'] = player_status['sp_max']
    t.sleep(0.5)
    print(f'SPが{sp_heal}回復！')
  t.sleep(1)
  if enemy_status["hp"] <= 0:
    battle_win()
    return
  print()
  is_defeated = enemy_act()
  if is_defeated:
    print(f'{player_status["name"]}は力尽きた！')
    t.sleep(1)
    battle_defeat()
  else:
    battle()

# スキルボタン
def press_skill(b):
  clear_output()
  print('スキルを選択してください')
  for i in range(len(player_status["skills"])):
    if i == cursor_locate:
      print('>',end='')
    else:
      print(' ',end='')
    print(f'SP{player_status["skills_sp"][i]}',end=' ')
    print(player_status["skills"][i])
  display(selectbutton)

# 逃げるボタン
def press_run(b):
  clear_output()
  if enemy_status["boss"]:
    clear_output()
    print('逃げられない！')
    t.sleep(1)
    battle()
    return
  print(f'{player_status["name"]}は逃げ出した！')
  t.sleep(0.5)
  if player_status["lv"] * r.randint(7,11)/10 >= enemy_status["lv"]/2:
    print('うまくいった！')
    t.sleep(1)
    battle_end()
  else:
    print('追いつかれてしまった！')
    t.sleep(1)
    print()
    is_defeated = enemy_act()
    if is_defeated:
      print(f'{player_status["name"]}は力尽きた！')
      t.sleep(1)
      battle_defeat()
    else:
      battle()

# 戻るボタン
def press_back(b):
  global scene
  cursor_locate = 0
  if scene == 'status':
    scene = 'dungeon'
    clear_output()
    dungeon_print()
  elif scene == 'battle':
    clear_output()
    battle()

# 決定ボタン
def press_enter(b):
  global enemy_status,player_status,cursor_locate,scene
  clear_output()
  # 戦闘中の処理
  if scene == 'battle':

    # カウンターの処理を行う
    if player_status["skills"][cursor_locate] == 'カウンター':
      player_status['def'] *= 2
      print(f'{player_status["name"]}は身構えた！')
      t.sleep(1)
      print()
      is_defeated = enemy_act()
      if is_defeated:
        print(f'{player_status["name"]}は力尽きた！')
        t.sleep(1)
        battle_defeat()
      else:
        if enemy_damage == 0:
          player_status['def'] /= 2
          print()
          print('カウンターに失敗した！')
          t.sleep(1)
          battle()
        else:
          player_damage = int((player_status["atk"]/4 + enemy_damage/2 - enemy_status["def"]/4) * r.randint(8,12) / 10)
          if player_damage < 1:
            player_damage = 1
          enemy_status["hp"] -= player_damage
          player_status['def'] /= 2
          print()
          print(f'{player_status["name"]}のカウンター！')
          t.sleep(0.5)
          print(f'{enemy_status["name"]}に{player_damage}のダメージ！')
          if player_status['sp_max'] > player_status['sp']:
            sp_heal = int(player_damage * 5 * r.randint(8,12) / 10)
            player_status["sp"] += sp_heal
            if player_status['sp_max'] <= player_status['sp']:
              player_status['sp'] = player_status['sp_max']
            t.sleep(0.5)
            print(f'SPが{sp_heal}回復！')
          t.sleep(1)
          if enemy_status["hp"] <= 0:
            battle_win()
            return
          else:
            battle()

    # ヒールの処理を行う
    elif player_status["skills"][cursor_locate] == 'ヒール' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      if player_status["hp"] == player_status["hp_max"]:
        print('HPは最大です')
        t.sleep(1)
        battle()
      else:
        player_status["sp"] -= player_status["skills_sp"][cursor_locate]
        heal = int(((player_status["lv"]+1) * 2 + 3) * r.randint(8,12) / 10)
        player_status["hp"] += heal
        if player_status['hp_max'] <= player_status['hp']:
          player_status['hp'] = player_status['hp_max']
        print('ヒールを唱えた！')
        t.sleep(0.5)
        print(f'HPが{heal}回復！')
        t.sleep(1)
        print()
        is_defeated = enemy_act()
        if is_defeated:
          print(f'{player_status["name"]}は力尽きた！')
          t.sleep(1)
          battle_defeat()
        else:
          battle()

    # 全力斬りの処理を行う
    elif player_status["skills"][cursor_locate] == '全力斬り' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      player_status["sp"] -= player_status["skills_sp"][cursor_locate]
      player_damage = int((player_status["atk"]/2 - enemy_status["def"]/4) * 1.4 * r.randint(8,12) / 10)
      if player_damage < 1:
        player_damage = 1
      enemy_status["hp"] -= player_damage
      print(f'{player_status["name"]}の全力斬り！')
      t.sleep(0.5)
      print(f'{enemy_status["name"]}に{player_damage}のダメージ！')
      t.sleep(1)
      if enemy_status["hp"] <= 0:
        battle_win()
        return
      print()
      is_defeated = enemy_act()
      if is_defeated:
        print(f'{player_status["name"]}は力尽きた！')
        t.sleep(1)
        battle_defeat()
      else:
        battle()

    # メガヒールの処理を行う
    elif player_status["skills"][cursor_locate] == 'メガヒール' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      if player_status["hp"] == player_status["hp_max"]:
        print('HPは最大です')
        t.sleep(1)
        battle()
      else:
        player_status["sp"] -= player_status["skills_sp"][cursor_locate]
        heal = int(((player_status["lv"]+1) * 4 + 3) * r.randint(8,12) / 10)
        player_status["hp"] += heal
        if player_status['hp_max'] <= player_status['hp']:
          player_status['hp'] = player_status['hp_max']
        print('メガヒールを唱えた！')
        t.sleep(0.5)
        print(f'HPが{heal}回復！')
        t.sleep(1)
        print()
        is_defeated = enemy_act()
        if is_defeated:
          print(f'{player_status["name"]}は力尽きた！')
          t.sleep(1)
          battle_defeat()
        else:
          battle()

    #ドラゴンスレイヤーの処理を行う
    elif player_status["skills"][cursor_locate] == 'ドラゴンスレイヤー' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      player_status["sp"] -= player_status["skills_sp"][cursor_locate]
      player_damage = int((player_status["atk"]/2 - enemy_status["def"]/4) * 1.8 * r.randint(8,12) / 10)
      if enemy_status["type"] == 'dra':
        player_damage *= 2
      if player_damage < 1:
        player_damage = 1
      enemy_status["hp"] -= player_damage
      print(f'{player_status["name"]}のドラゴンスレイヤー！')
      t.sleep(0.5)
      print(f'{enemy_status["name"]}に{player_damage}のダメージ！')
      t.sleep(1)
      if enemy_status["hp"] <= 0:
        battle_win()
        return
      print()
      is_defeated = enemy_act()
      if is_defeated:
        print(f'{player_status["name"]}は力尽きた！')
        t.sleep(1)
        battle_defeat()
      else:
        battle()

    # SPが足りない時の出力をする
    else:
      print('SPが足りません')
      t.sleep(1)
      battle()

    cursor_locate = 0

  # ステータス画面の処理
  elif scene == 'status':

    # ヒールの処理を行う
    if player_status["skills"][cursor_locate] == 'ヒール' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      if player_status["hp"] == player_status["hp_max"]:
        print('HPは最大です')
        t.sleep(1)
      else:
        player_status["sp"] -= player_status["skills_sp"][cursor_locate]
        heal = int(((player_status["lv"]+1) * 2 + 3) * r.randint(8,12) / 10)
        player_status["hp"] += heal
        if player_status['hp_max'] <= player_status['hp']:
          player_status['hp'] = player_status['hp_max']
        print('ヒールを唱えた！')
        t.sleep(0.5)
        print(f'HPが{heal}回復！')
        t.sleep(1)

    # メガヒールの処理を行う
    elif player_status["skills"][cursor_locate] == 'メガヒール' and player_status["skills_sp"][cursor_locate] <= player_status["sp"]:
      clear_output()
      if player_status["hp"] == player_status["hp_max"]:
        print('HPは最大です')
        t.sleep(1)
      else:
        player_status["sp"] -= player_status["skills_sp"][cursor_locate]
        heal = int(((player_status["lv"]+1) * 4 + 3) * r.randint(8,12) / 10)
        player_status["hp"] += heal
        if player_status['hp_max'] <= player_status['hp']:
          player_status['hp'] = player_status['hp_max']
        print('メガヒールを唱えた！')
        t.sleep(0.5)
        print(f'HPが{heal}回復！')
        t.sleep(1)

    # その他のスキルの処理をする
    else:
      clear_output()
      print('使っても効果がありません')
      t.sleep(1)

    press_status(b)

  # ゲームを開始するときの出力を行う
  else:
    if name_input.value == '':
      print('                 -Python RPG-')
      print('         ～魔術師リファクタの大迷宮～')
      print()
      print('          !名前が入力されていません!')
      print()
      display(name_input)
      display(startbutton)
    elif len(name_input.value) > 6:
      print('                 -Python RPG-')
      print('         ～魔術師リファクタの大迷宮～')
      print()
      print('        !名前を6文字以下にしてください!')
      print()
      display(name_input)
      display(startbutton)
    else:
      player_status["name"] = name_input.value
      scene = 'dungeon'
      for i in range(15):
        for j in range(15):
          print('@',end = ' ')
        print()
      t.sleep(1)
      clear_output()
      dungeon_set()
      enemy_set()
      dungeon_print()

# 名前を入力するテキストボックスを定義する
name_input = widgets.Text(placeholder='名前を入力(6文字まで)',description='NAME',)

up.on_click(press_up)
down.on_click(press_down)
left.on_click(press_left)
right.on_click(press_right)
attack.on_click(press_attack)
skill.on_click(press_skill)
run.on_click(press_run)
back.on_click(press_back)
enter.on_click(press_enter)
status.on_click(press_status)
print('                 -Python RPG-')
print('         ～魔術師リファクタの大迷宮～')
t.sleep(2)
print()
display(name_input)
display(startbutton)

                 -Python RPG-
         ～魔術師リファクタの大迷宮～



Text(value='', description='NAME', placeholder='名前を入力(6文字まで)')